In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import json
import os
import shutil
import random
from PIL import Image

In [2]:
# Définir les transformations pour les données
transform_couleur = transforms.Compose([
    transforms.Resize((112, 112)),  # Redimensionner les images à 112x112
    transforms.ToTensor(),  # Convertir les images en tenseurs PyTorch
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normaliser avec les statistiques de l'ImageNet
])

# Définir une nouvelle transformation pour convertir les images en niveaux de gris
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Redimensionner les images à 112x112
    transforms.Grayscale(),  # Convertir les images en niveaux de gris
    transforms.ToTensor(),  # Convertir les images en tenseurs PyTorch
])

# Télécharger et charger le jeu de données Flowers102
dataset = datasets.Flowers102(root='./data', 
                              split='train', 
                              transform=transform_couleur, 
                              download=True)

# Définir les proportions pour l'entraînement et le test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Diviser dataset1 en ensemble d'entraînement et de test
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [3]:
print("nombre d'échantillons d'entraînement: " + str(len(train_dataset)) + "\n" +
      "nombre d'échantillons de test: " + str(len(test_dataset)))

nombre d'échantillons d'entraînement: 816
nombre d'échantillons de test: 204


In [4]:
print("type de données du 1er échantillon d'entraînement: ", train_dataset[0][0].type())
print("taille du 1er échantillon d'entraînement: ", train_dataset[0][0].size())

type de données du 1er échantillon d'entraînement:  torch.FloatTensor
taille du 1er échantillon d'entraînement:  torch.Size([3, 112, 112])


In [5]:
# Créer les DataLoader pour itérer sur les jeux de données
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Définir le modèle CNN

### Aplati pour 112 * 112 * 3

In [12]:

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=102):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.fc1 = nn.Linear(32 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Aplatir
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=1)
        return x

model = SimpleCNN(num_classes=102)


In [13]:
# Définition de l'optimiseur et de la perte d'entropie croisée
optimiseur = torch.optim.Adam(model.parameters(), lr=0.001)
critere = nn.CrossEntropyLoss()

In [ ]:
epochs = 25
Perte = []
acc = []


for epoque in range(epochs):
    model.train()
    running_loss = 0.0
    
    for images, etiquettes in train_loader:
        optimiseur.zero_grad()
        sorties = model(images)
        perte = critere(sorties, etiquettes)
        perte.backward()
        optimiseur.step()
        running_loss += perte.item()
    
    Perte.append(running_loss / len(train_loader))
    
    correct = 0
    total = 0
    model.eval()
    
    with torch.no_grad():
        for images, etiquettes in test_loader:
            sorties = model(images)
            _, predit = torch.max(sorties.data, 1)
            total += etiquettes.size(0)
            correct += (predit == etiquettes).sum().item()
    
    precision = 100 * correct / total
    acc.append(precision)
    
    if epoque % 5 == 0:
        print(f'Époque : {epoque}, Perte : {running_loss / len(train_loader)}, Précision : {precision}')

print(f'Nombre d\'images dans le dataset original: {len(dataset)}')
print(f'Nombre d\'images dans le train_loader: {len(train_loader.dataset)}')
print(f'Nombre d\'images dans le test_loader: {len(test_loader.dataset)}')


In [ ]:
plt.plot(Perte)
plt.xlabel("nombre d'époques")
plt.ylabel("perte totale")
plt.title("Perte")
plt.show()

In [ ]:
plt.plot(acc)
plt.xlabel("nombre d'époques")
plt.ylabel("précision totale")
plt.title("Précision")
plt.show()

In [ ]:
import torch
import torchvision.models as models

# Définir des données factices en entrée
dummy_input = torch.randn(1, 3, 112, 112)  # Les dimensions correspondent à celles des données MNIST

# Spécifier le chemin de sauvegarde du modèle ONNX
onnx_path = "flower_model.onnx"

# Exporter le modèle en format ONNX
torch.onnx.export(model, dummy_input, onnx_path)